# Project 1: Pricing Derivatives
In this Project I implement and test the Black-Scholes formulae for pricing various options. Then I perform various consistency checks on my implementation. 

Prices will be a function of time-to-maturity $T$, continuously compounding rate $r$, dividend rate $d$, strike $K$, spot $S$. and volatility $\sigma$
### Zero-Coupon Bond
The price of a zero-coupon bond is 
$$P = e^{-rT}$$
### Forward Contract
The price of a forward contract is $$ P = e^{-rT}\left(e^{(r-d)T}S-K\right)$$

### Call and Put Options
The prices of Call and Put options are respectively
$$C = Se^{-dT}N(d_1) - K e^{-rT} N(d_2),$$
$$P = K e^{-rT} N(-d_2) - Se^{-dT}N(-d_1),$$
where 
$$d_1 = \frac{\log(\frac{S}{K}) + T(r-d+\frac{\sigma^2}{2})}{\sigma \sqrt{T}},$$
$$d_2 = d_1 - \sigma \sqrt{T},$$
and $N(x)$ is the standard normal cumulative distribution function:
$$N(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^x e^{\frac{-t^2}{2}}dt.$$

### Digital Calls and Puts
The prices of digital calls and p
uts are respectively
$$C = e^{-rT}N(d_2), $$
$$P = e^{-rT}N(-d_2).$$